In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drugsComTrain_raw.csv')

In [ ]:
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [ ]:
df.drugName.unique().shape

(3436,)

In [ ]:
df.condition.unique().shape

(885,)

### Content Based

In [ ]:
df.drop_duplicates(subset=['review'],inplace=True)

In [ ]:
# Creates a column that contains a combintion of keywords,cast,director and genres
original_df = df.copy()
df = df.sample(n=15000, replace=True,random_state=1289)
df = df.reset_index()
df['soup'] = df['review']+df['condition']
df.dropna(axis=0, inplace=True)

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer,TfidfVectorizer, CountVectorizer
# Vectorizes words to numbers and builds a sparse matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['soup'])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
#Computes similarity between movies using cosine similarity metric
cosine_sim = cosine_similarity(count_matrix, count_matrix)

### Colaborative Recommendation

In [ ]:
#df = df.reset_index()
titles = df[['condition','drugName']]
series_tmp=pd.Series
#series_tmp = series_tmp.append(pd.Series(df.index,index=[(0,'a')]))
indices = pd.Series(df.index, index=[df['condition'],df['drugName']])

In [ ]:
indices

condition            drugName                          
Diabetes, Type 2     Trulicity                                 0
Weight Loss          Phentermine                               1
Migraine             Acetaminophen / aspirin / caffeine        2
Pseudotumor Cerebri  Acetazolamide                             3
ADHD                 Metadate CD                               4
                                                           ...  
Birth Control        Etonogestrel                          14995
Oophorectomy         Estradiol Patch                       14996
Atrial Fibrillation  Propafenone                           14997
Anxiety              Clonazepam                            14998
Chronic Pain         Oxycodone                             14999
Length: 14927, dtype: int64

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from surprise import Reader, Dataset, SVD
#Define a Reader object-The Reader object helps in parsing the file or dataframe containing ratings
reader = Reader()

In [ ]:
#df_content=df
#df=original_df
# label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['drugId']= le.fit_transform(df['drugName'])
df['condId']= le.fit_transform(df['condition'])

In [ ]:
reader = Reader()
#Create dataset to be used for building the filter
data = Dataset.load_from_df(df[['uniqueID', 'drugId','rating']], reader)

In [ ]:
data

In [ ]:
#Define the algorithm in this case SVD
svd = SVD()
#Evaluate performance in terms of RMSE
from surprise.model_selection import cross_validate
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.8065  3.8109  3.8398  3.8084  3.8226  3.8177  0.0124  
MAE (testset)     3.5096  3.5201  3.5472  3.5057  3.5299  3.5225  0.0149  
Fit time          2.54    2.61    2.49    2.23    1.65    2.30    0.35    
Test time         0.05    0.04    0.03    0.03    0.02    0.03    0.01    


{'fit_time': (2.538142442703247,
  2.6102468967437744,
  2.4898107051849365,
  2.2292284965515137,
  1.6501669883728027),
 'test_mae': array([3.50959957, 3.52007917, 3.54715136, 3.50569162, 3.5298982 ]),
 'test_rmse': array([3.80650943, 3.81092894, 3.83982151, 3.80840143, 3.82262618]),
 'test_time': (0.04830050468444824,
  0.035994529724121094,
  0.03032398223876953,
  0.03301668167114258,
  0.017048120498657227)}

In [ ]:
## perform training and testing on the trainset
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from surprise import accuracy

trainset, testset = train_test_split(data, test_size=0.2)
predictions = svd.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 3.7994


3.7993815720242305

In [ ]:
df

,index,uniqueID,drugName,condition,review,rating,date,usefulCount,soup,drugId,condId
0,9053,11860,Trulicity,"Diabetes, Type 2","""Been on trulicity for 6 months. No diarrhea, ...",4,15-Dec-15,24,"""Been on trulicity for 6 months. No diarrhea, ...",1536,154
1,30311,128799,Phentermine,Weight Loss,"""I Started 37.5mg on November 22, 2014 went to...",10,28-Dec-14,65,"""I Started 37.5mg on November 22, 2014 went to...",1193,493
2,148263,205303,Acetaminophen / aspirin / caffeine,Migraine,"""I have occasional migraines and I can only ge...",10,9-Dec-14,10,"""I have occasional migraines and I can only ge...",10,298
3,9993,169211,Acetazolamide,Pseudotumor Cerebri,"""I&#039;ve been taking this medication regular...",1,6-Mar-17,5,"""I&#039;ve been taking this medication regular...",26,400
4,59722,223901,Metadate CD,ADHD,"""My son is 7 and has been on this medicine for...",9,2-Oct-09,29,"""My son is 7 and has been on this medicine for...",976,27
...,...,...,...,...,...,...,...,...,...,...,...
14995,150659,144769,Etonogestrel,Birth Control,"""I&#039;ve been on the nexplanon since May 201...",5,28-Jan-16,1,"""I&#039;ve been on the nexplanon since May 201...",602,85
14996,16122,114588,Estradiol Patch,Oophorectomy,"""I have been on the patch for a month now and ...",8,9-Jun-09,35,"""I have been on the patch for a month now and ...",586,337
14997,39690,163199,Propafenone,Atrial Fibrillation,"""59 yr old m with afib since I was in my 30s. ...",10,8-Nov-16,5,"""59 yr old m with afib since I was in my 30s. ...",1251,66
14998,94227,172844,Clonazepam,Anxiety,"""Started this medicine a month ago and I have ...",10,8-Apr-14,99,"""Started this medicine a month ago and I have ...",358,58


In [ ]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df1 = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df1['Iu'] = df1.uid.apply(get_Iu)
df1['Ui'] = df1.iid.apply(get_Ui)
df1['err'] = abs(df1.est - df1.rui)
best_predictions = df1.sort_values(by='err')[:100]
worst_predictions = df1.sort_values(by='err')[-10:]

In [ ]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
3544,129239,1577,5.0,5.0,{'was_impossible': False},0,1854,0.0
4415,35150,1575,5.0,5.0,{'was_impossible': False},0,201,0.0
1158,78932,1620,5.0,5.0,{'was_impossible': False},0,16,0.0
15394,1174,2629,5.0,5.0,{'was_impossible': False},0,156,0.0
22090,101774,1770,5.0,5.0,{'was_impossible': False},0,86,0.0
...,...,...,...,...,...,...,...,...
15186,226272,1080,5.0,5.0,{'was_impossible': False},0,1632,0.0
11321,125853,2935,5.0,5.0,{'was_impossible': False},0,182,0.0
21188,34726,1819,5.0,5.0,{'was_impossible': False},0,175,0.0
18830,47579,2672,5.0,5.0,{'was_impossible': False},0,123,0.0


### Hybrid Recommender

In [ ]:
id_map=df[['uniqueID','drugId','drugName']]
#id_map['drugId'] = id_map['drugName'].apply(convert_int)
id_map.columns = ['uniqueID','drugId','drugName']
id_map = id_map.merge(df[['condition', 'uniqueID']], on='uniqueID').set_index('condition')

In [ ]:
id_map.drop_duplicates(inplace=True)

In [ ]:
indices_map = id_map.set_index('uniqueID')

In [ ]:
def hybrid(userId,drugName,condition):
    
    #Extract index of movie tilte
    idx = indices[condition,drugName]
    #Extract TMDB id of the movie
    uId = id_map.loc[condition]['uniqueID']
    
    #Extract movie id internally assigned by the dataset
    drugid = id_map.loc[condition]['drugId']
    
    #Extract the similarity scores and their corresponding index for every movie from the cosine similarity matrix
    #print(idx)
    sim_scores = list(enumerate(cosine_sim[int(idx[0])]))
    #Sort the (index,score) tuples in decreasing order of similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #Select top 25
    sim_scores = sim_scores[1:26]
    #Store the cosine_sim indices of the top 25 movies in a list
    drug_indices = [i[0] for i in sim_scores]
    
    #Extract metadata of the movie
    movies = df.iloc[drug_indices][['drugName','drugId', 'rating', 'uniqueID','condition']]
    #print(movies)
    #Compute predicted ratings using SVD
    movies['est'] = movies['uniqueID'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['drugId']).est)
    
    #Sort the movies in decreasing order of predicted rating 
    movies = movies.sort_values('est', ascending=False)
    
    #Return top 10 movies as recommendations
    return movies.head(10)

In [ ]:
hybrid(62696,'Citalopram','Depression')


/usr/local/lib/python3.7/dist-packages/ipykernel/ipkernel.py:208: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


,drugName,drugId,rating,uniqueID,condition,est
7782,Escitalopram,575,9,214651,Depression,5
14476,Prozac,1262,7,181250,Depression,5
14833,Antabuse,116,10,120812,Alcohol Dependence,5
14009,Venlafaxine,1582,9,121210,Depression,5
8240,Venlafaxine,1582,9,121210,Depression,5
5341,Adderall,42,10,88382,ADHD,5
558,Adderall,42,10,88382,ADHD,5
8196,Bupropion,246,6,225022,Anxiety,5
9849,Scopolamine,1349,1,175412,Nausea/Vomiting,5
14173,Lisinopril,900,1,32241,High Blood Pressure,5
